In [1]:
import pandas as pd
from konlpy.tag import Mecab
from functools import *
from collections import Counter
import numpy as np
from tqdm.auto import tqdm

In [2]:
def get_word(res):
    res_w = []
    for word in res:
        if len(word) < 2:
            continue
        else:
            res_w.append(word)
    return res_w

tokenizer = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic")

def split_word_df(df_trg):
    res_list = []
    print(f'total : {len(df_trg)}')
    for content in tqdm(df_trg.contents_clean):
        res = tokenizer.nouns(content)
        res = get_word(res)
        res_list.append(res)
    df_trg['words'] = res_list
    return df_trg

def get_word_pct_dict(lst_word , df_src , trg_col):
    word_pct_dict = {}
    for word in tqdm(lst_word):
        word_pct_dict.update(  {word : len(df_src[df_src[trg_col].apply(lambda x: word in x)])/len(df_src)} )
        sorted_word_pct_dict = sorted(word_pct_dict.items(), key=lambda x:x[1], reverse=True)
    return sorted_word_pct_dict

def flattened_list(lst):
    flattened_lst = [item for sublist in lst for item in sublist]
    return flattened_lst

In [3]:
df = pd.read_parquet('../df_qesg28_final_time.pq')

# 전체 기준으로 단어 빈도 

In [4]:
df_temp = split_word_df(df)

total : 106164


  0%|          | 0/106164 [00:00<?, ?it/s]

In [6]:
df_temp= df_temp.reset_index(drop= False)
df_temp.to_parquet('df_temp.pq')

# Load Data

In [7]:
df_temp = pd.read_parquet('df_temp.pq')

In [12]:
# loop 로 전체 코퍼스 만들기
corpus = []
for val in tqdm(df_temp.words):
    corpus += list(val)

corpus= list(set(corpus))
print(len(corpus))

  0%|          | 0/106164 [00:00<?, ?it/s]

99688


In [ ]:
res = get_word_pct_dict(corpus ,df_temp, 'words' )

  0%|          | 0/99688 [00:00<?, ?it/s]

In [ ]:
df_src = df_temp
trg_col = 'words'
df_src[trg_col].apply(lambda x: word in x)

In [ ]:
df_temp

In [ ]:
# 전체에서 90% 이상 나오는 단어는 제거 
df_trg = df_temp
# 단어 리스트 
word_set = []
for i , row in tqdm(df_trg.iterrows()):
    word_set += list(set(row.words))
word_set = list(set(word_set))
    

In [ ]:
print(len(word_set))
count_mat = np.zeros_like(word_set , dtype = 'int32')
for i , word in tqdm(enumerate(word_set)):
    for k , row in df_trg.iterrows():
        if word in row.words:
            count_mat[i] += 1 

0it [00:00, ?it/s]

99688


0it [00:00, ?it/s]

In [ ]:
word_set

# 특정 카테고리 기준으로 단어 빈도 

In [3]:
df_sm = df.loc[df.contents_clean != '']\
    .loc[df.y != '제외']\
    .loc[df.y != '기타']\
    .loc[df.time < '2021-04-28']\
    .loc[df.time > '2020-01-01']

In [4]:
pd.unique(df_sm.y)
df_sm.y.value_counts()

에너지및온실가스       7708
사회공헌           6326
인적자원관리         5741
소유구조           4898
ESG거버넌스        2867
제품/서비스         2208
공급망관리          2044
사업장안전및보건       1704
내부통제/투명성       1492
소비자보호          1449
이사회구성과운영       1377
지역사회영향         1373
자원순환           1232
공정거래           1173
계열회사와의거래        519
주주가치보호          463
이익배분            416
감사기구            375
인권              263
환경관리체계          202
대기오염            167
공시              118
친환경제품/서비스개발      79
수질오염             79
생물다양성            49
화학물질배출           34
기타오염             22
토양오염              6
Name: y, dtype: int64

In [5]:
df_trg = df.loc[df.y == '환경관리체계']

In [29]:
df_trg.to_excel('환경관리체계.xlsx') 
df_trg = pd.read_excel('공시.xlsx', index_col= 0)

# 토크나이저 

In [25]:
# 전체에서 90% 이상 나오는 단어는 제거 

# 단어 리스트 
word_set = []
for i , row in df_trg.iterrows():
    word_set += list(set(row.words))

word_set = list(set(word_set))
    
print(len(word_set))
count_mat = np.zeros_like(word_set , dtype = 'int32')
for i , word in tqdm(enumerate(word_set)):
    for k , row in df_trg.iterrows():
        if word in row.words:
            count_mat[i] += 1 
        

    

4550


0it [00:00, ?it/s]

In [26]:
doc_count = len(df_trg) # total doc count 
ratio_mat = count_mat / doc_count

In [27]:
df_ratio = pd.DataFrame()
df_ratio['word']  = word_set
df_ratio['ratio'] = ratio_mat


In [37]:
df_ratio_top = df_ratio.loc[df_ratio.ratio >= 0.8* df_ratio.ratio.max() ]
print(0.8* df_ratio.ratio.max())

0.5320574162679427


In [29]:
df_ratio_cuttop = df_ratio.loc[df_ratio.ratio < 0.9* df_ratio.ratio.max() ]

In [36]:
df_ratio_top

,word,ratio
876,기업,0.583732
2734,환경,0.631579
3383,사업,0.626794
4374,친환경,0.665072


In [24]:
df_ratio_cuttop.sort_values(by = 'ratio', ascending=False)

,word,ratio
20186,기업,0.583732
1039,기업,0.583732
2332,기업,0.583732
9724,기업,0.583732
17136,기업,0.583732
...,...,...
19613,급등세,0.004785
5679,당사자,0.004785
19615,비엠,0.004785
19616,바이,0.004785


In [73]:
# 고빈도 저빈도 제거 - 리스트 

# 단어 리스트 
word_list = []
for i , row in df_trg.iterrows():
    word_list += list(row.words)    
res = dict(Counter(word_list))
df_count = pd.DataFrame.from_dict(res,  orient='index')
df_count['word']  = df_count.index
df_count.columns = ['count','word']
df_fromlist = df_count.sort_values(by = 'count' , ascending=False)

In [74]:
# 고빈도 저빈도 제거 - 리스트 

# 단어 리스트 
word_set = []
for i , row in df_trg.iterrows():
    word_set += list(set(row.words))

res = dict(Counter(word_set))

df_count = pd.DataFrame.from_dict(res,  orient='index')
df_count['word']  = df_count.index
df_count.columns = ['count','word']

df_fromset = df_count.sort_values(by = 'count' , ascending=False)

In [ ]:
# 전체에서 각 단어들이 몇번이나 나왔는지. 
# 이후 별로 안나오는 것들은 뺴버린다 .
# 잘나오는 것들도 빼버린다. 




